In [1]:
import pandas as pd

In [2]:
data_latih = pd.read_csv('./output/data-clean-en.csv', encoding='utf-8')
data_latih[['resultStemming', 'labels']]

,resultStemming,labels
0,"['chatgpt', 'optim', 'languag', 'model', 'dial...",neutral
1,"['tri', 'talk', 'chatgpt', 'new', 'ai', 'syste...",good
2,"['chatgpt', 'optim', 'languag', 'model', 'dial...",neutral
3,"['thrill', 'share', 'chatgpt', 'new', 'model',...",good
4,"['minut', 'ago', 'releas', 'new', 'chatgpt', '...",bad
...,...,...
219289,"['softwar', 'project', 'tri', 'replic', 'chatg...",bad
219290,"['ask', 'chatgpt', 'write', 'nye', 'joke', 'se...",good
219291,"['chatgpt', 'disassembl', 'dissembl']",bad
219292,"['predict', 'chatgpt', 'noth', 'realli', 'spec...",bad


In [3]:
data_uji = pd.read_csv('./output/data-clean-ind.csv', encoding='utf-8')
data_uji[['resultStemming']]

,resultStemming
0,"['kawankawan', 'bingung', 'asingdengan', 'isti..."
1,"['terimakasih', 'jelas', 'bantu']"
2,"['tolong', 'pin']"
3,"['ralat', 'bang', 'poin', 'chat', 'gpt', 'fine..."
4,"['thanks', 'additional', 'info', 'mas']"
...,...
1236,"['ngga', 'paham', 'blas']"
1237,"['dr', 'indra', 'bsa', 'ngoding', 'ga']"
1238,['t']
1239,['pohara']


### TF-IDF

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

#### TF-IDF data latih

In [5]:
vectorizer = TfidfVectorizer()

X_latih = vectorizer.fit_transform(data_latih['resultStemming'])

tfidf_sum = np.array(X_latih.sum(axis=0)).ravel()

tfidf_df = pd.DataFrame({
    'word': vectorizer.get_feature_names_out(),
    'tfidf_score': tfidf_sum
})

print(tfidf_df)


                               word  tfidf_score
0                                __     1.227053
1                               ___     2.478431
2                              ____     3.088597
3                             _____     5.026893
4                            ______     3.753742
...                             ...          ...
285473                        𝙬𝙤𝙧𝙧𝙮     0.309286
285474                          𝙮𝙤𝙪     0.260824
285475                   𝚊𝚛𝚝𝚒𝚏𝚒𝚌𝚒𝚊𝚕     0.433232
285476  𝚒𝚗𝚝𝚎𝚕𝚕𝚒𝚐𝚎𝚗𝚌𝚎nhttpstcowhuujo     0.433232
285477                  𝚒𝚗𝚝𝚎𝚛𝚟𝚒𝚎𝚠𝚎𝚍     0.433232

[285478 rows x 2 columns]


#### TF-IDF data uji

In [6]:
X_uji = vectorizer.fit_transform(data_uji['resultStemming'])

tfidf_df = pd.DataFrame({
    'word': vectorizer.get_feature_names_out(),
    'tfidf_score': X_uji.toarray().sum(axis=0)
})

print(tfidf_df)

         word  tfidf_score
0      aamiin     1.000000
1        abal     0.310682
2         abc     0.262327
3        abdi     0.092280
4       about     0.700067
...       ...          ...
3682    zaman     2.526454
3683  zaremba     0.312446
3684      zat     0.278978
3685     zero     0.876446
3686     zlrr     0.453418

[3687 rows x 2 columns]


### Oversampling

In [7]:
print("Sebelum Oversampling, jumlah label 'POSITIF': {}".format(sum(data_latih['labels'] == 'good')))
print("Sebelum Oversampling, jumlah label 'NEGATIF': {} ".format(sum(data_latih['labels'] == 'bad')))
print("Sebelum Oversampling, jumlah label 'NETRAL': {} \n".format(sum(data_latih['labels'] == 'neutral')))

Sebelum Oversampling, jumlah label 'POSITIF': 56011
Sebelum Oversampling, jumlah label 'NEGATIF': 107796 
Sebelum Oversampling, jumlah label 'NETRAL': 55487 



#### SMOTE

In [8]:
# SMOTE
# !pip install imblearn

from imblearn.over_sampling import SMOTE

In [9]:
sm = SMOTE(random_state=2)

X_res, y_res = sm.fit_resample(X_latih, data_latih['labels'])

KeyError: 'labels'

In [12]:
print("Sebelum Oversampling, jumlah label 'POSITIF': {}".format(sum(y_res == 'good')))
print("Sebelum Oversampling, jumlah label 'NEGATIF': {} ".format(sum(y_res == 'bad')))
print("Sebelum Oversampling, jumlah label 'NETRAL': {} \n".format(sum(y_res == 'neutral')))

Sebelum Oversampling, jumlah label 'POSITIF': 107796
Sebelum Oversampling, jumlah label 'NEGATIF': 107796 
Sebelum Oversampling, jumlah label 'NETRAL': 107796 



### KNN Model

In [13]:
from sklearn.neighbors import KNeighborsClassifier

In [14]:
model = KNeighborsClassifier()

#### K-Fold Cross Validation

In [15]:
from sklearn.model_selection import cross_val_score

In [16]:
scores = cross_val_score(model, X_res, y_res, cv=10)

print("Cross-validation scores: ", scores)
print("Average cross-validation score: ", scores.mean())

Cross-validation scores:  [0.55867528 0.56603482 0.58177433 0.58622716 0.58430997 0.69646557
 0.71718359 0.71245246 0.7131548  0.69766219]
Average cross-validation score:  0.6413940159466758
